# Binarized NN for MNIST

In [5]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from binarized_modules import  BinarizeLinear,BinarizeConv2d

In [6]:
torch.manual_seed(1);

In [7]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/.pytorch/MNIST_data/', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/.pytorch/MNIST_data/', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True)

In [8]:
class MNIST_compressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 1, 4, stride = 4)
        self.conv1.weight.data.fill_(1/16)
        self.conv1.bias.data.fill_(0)
    def forward(self, x):
        m = nn.ConstantPad2d(2, -1)
        x = self.conv1(m(x))
        return x

In [50]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.infl_ratio = 3
        self.fc1 = BinarizeLinear(64, 1000*self.infl_ratio)
        self.htanh1 = nn.Hardtanh()
        self.bn1 = nn.BatchNorm1d(1000 * self.infl_ratio)
        self.fc2 = BinarizeLinear(1000*self.infl_ratio, 1000*self.infl_ratio)
        self.htanh2 = nn.Hardtanh()
        self.bn2 = nn.BatchNorm1d(1000 * self.infl_ratio)
        self.logsoftmax = nn.LogSoftmax()
        
    def forward(self, x):
        x = x.view(-1, 64)
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.htanh1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.htanh2(x)
        return self.logsoftmax(x)

In [51]:
model = Net()
compressor = MNIST_compressor()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [52]:
def train(epoch):
    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = compressor(images).detach()
        images, labels = Variable(images), Variable(labels)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        optimizer.zero_grad()
        loss.backward()
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.data.copy_(p.org)
        optimizer.step()
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.org.copy_(p.data.clamp_(-1,1))
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(images), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [53]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for (images, labels) in test_loader:
        images = compressor(images).detach()
        images, labels = Variable(images, volatile=True), Variable(labels)
        output = model(images)
        test_loss = criterion(output, labels).data[0]
        prediction = output.data.max(1, keepdim=True)[1]
        correct += prediction.eq(labels.data.view_as(prediction)).sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [54]:
for epoch in range(1):
    train(epoch)
    test()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 0 [0/60000 (0%)]	Loss: 8.245426
Train Epoch: 0 [6400/60000 (11%)]	Loss: 6.064167
Train Epoch: 0 [12800/60000 (21%)]	Loss: 6.024046
Train Epoch: 0 [19200/60000 (32%)]	Loss: 6.020351
Train Epoch: 0 [25600/60000 (43%)]	Loss: 6.031588
Train Epoch: 0 [32000/60000 (53%)]	Loss: 6.021167
Train Epoch: 0 [38400/60000 (64%)]	Loss: 6.021582
Train Epoch: 0 [44800/60000 (75%)]	Loss: 6.023191
Train Epoch: 0 [51200/60000 (85%)]	Loss: 6.023415
Train Epoch: 0 [57600/60000 (96%)]	Loss: 6.022345


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if __name__ == '__main__':



Test set: Average loss: 0.0006, Accuracy: 3009/10000 (30%)

